In [1]:
import requests
import json
import xmltodict
import urllib.parse
import opencitingpy
import datetime
import sqlite3
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
import re
import math
import ipywidgets as widgets
from IPython.display import display, clear_output
import re 
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from collections import Counter
from IPython.display import display
from Bio import Entrez
from matplotlib.ticker import MaxNLocator
import scipy
from scipy import stats
import mercury as mr
from mycolorpy import colorlist as mcp

In [2]:
# Anlegen von Datenbankspeicherort, falls er nicht existiert
path = "C:\MA_Pethke_3992454"
if not os.path.exists(path):
    mr.Markdown(f"## Es liegen noch keine Daten vor.")

# Verbindung zu DB aufbauen, DB wird automatisch erstellt, wenn keine da ist
con = sqlite3.connect("C:\MA_Pethke_3992454\MA_3992454_jp.db")
cur = con.cursor()

In [3]:
## Funktionen um Daten in DB einzupflegen

# Dokument Daten
def DB_Insert_Dokument(PMID, Titel, Jahr_Dok):
    try:
        cur.execute("INSERT INTO Dokument(PMID, Titel, Jahr_Dok) VALUES ('"+str(PMID)+"', '"+str(Titel)+"', '"+str(Jahr_Dok)+"')")
        con.commit()
        # Insert erfolgreich
    except:
        # Dokumenten-Daten schon in DB
        return
    
# Zitationsdaten für ein Dokument
def DB_Insert_Zitationen(values_str):
    try:
        cur.execute("INSERT INTO Zitationen(PMID, Jahr, Anzahl) VALUES " + str(values_str))
        con.commit()
        # Insert erfolgreich
    except:
        # Zitationsdaten schon in DB
        return
    
# MeSH Daten
def DB_Insert_MeSH_Daten(UI, Term, Baum, Jahr_MeSH):
    try:
        cur.execute("INSERT INTO MeSH(UI, Term, Baum, Jahr_MeSH) VALUES ('"+str(UI)+"', '"+str(Term)+"', '"+str(Baum)+"', '"+str(Jahr_MeSH)+"')")
        con.commit()
        # Insert erfolgreich
    except:
        # MeSH schon in DB
        return
    
# MeSH Analyse-Daten 
def DB_Insert_MeSH_Analyse(EK, NEK, NEKK, UI, prozent):
    try:
        # print("UPDATE MeSH SET EK = '"+str(EK)+"', NEK = '"+str(NEK)+"', NEKK = '"+str(NEKK)+"' WHERE UI ='" + str(UI) + "'")
        cur.execute("UPDATE MeSH SET EK = '"+str(EK)+"', NEK = '"+str(NEK)+"', NEKK = '"+str(NEKK)+"', EK_NEK = '" + str(prozent) + "' WHERE UI ='" + str(UI) + "'")
        con.commit()
        # Insert erfolgreich
    except:
        # MeSH Analysedaten schon in DB
        return    
    
# MeSH-Dokumenten Zuordnung
def DB_Insert_MeSH_PMID(UI, PMID):
    try:
        cur.execute("INSERT INTO MeSH_Dok(UI, PMID) VALUES ('"+str(UI)+"', '"+str(PMID)+"')")
        con.commit()
        # Insert erfolgreich
    except:
        # MeSH Analysedaten schon in DB
        return    
    
# MeSH-Dokument Analyse-Daten 
def DB_Insert_MeSH_Dokument_Analyse(EK_Dok, NEK_Dok, NEKK_Dok, UI, PMID, prozent):
    try:
        # print("UPDATE MeSH_Dok SET EK_Dok = '"+str(EK_Dok)+"', NEK_Dok = '"+str(NEK_Dok)+"', NEKK_Dok = '"+str(NEKK_Dok)+"' WHERE UI = '"+str(UI)+"' and PMID = '"+str(PMID)+"'")
        cur.execute("UPDATE MeSH_Dok SET EK_Dok = '"+str(EK_Dok)+"', NEK_Dok = '"+str(NEK_Dok)+"', NEKK_Dok = '"+str(NEKK_Dok)+"', EK_NEK_Dok = '" + str(prozent) + "' WHERE UI = '"+str(UI)+"' and PMID = '"+str(PMID)+"'")
        con.commit()
        # Insert erfolgreich
    except:
        # MeSH Analysedaten schon in DB
        return
    

def DB_Select_Alle_MeSH():
    cur.execute("SELECT Term FROM MeSH")
    data = cur.fetchall()
    term = [x[0] for x in data]
    return term

def DB_Select_Alle_Dokumente_MeSH(meshUI):
    cur.execute("SELECT Titel, Dokument.PMID FROM Dokument JOIN MeSH_Dok ON Dokument.PMID = MeSH_Dok.PMID WHERE UI = '" + str(meshUI) +"'")
    output = cur.fetchall()
    titles = [x[0] for x in output]
    json_list={}
    for title, pmid in output:
        json_list[title] = pmid
    return json_list, titles

# def DB_Select_PMID_fromTitel(titel):
#     cur.execute("SELECT PMID FROM Dokument WHERE Titel = '" + str(titel) +"'")
#     return cur.fetchall()[0][0]

def DB_Select_Jahr_MeSH(UI):
    cur.execute("SELECT Jahr_MeSH FROM MeSH WHERE UI = '" + str(UI)+ "'")
    return cur.fetchall()[0][0]

def DB_Select_Term(UI):
    cur.execute("SELECT Term FROM MeSH WHERE UI = '" + str(UI)+ "'")
    return cur.fetchall()[0][0]

def DB_Select_Jahr_Dok(PMID):
    cur.execute("SELECT Jahr_Dok FROM Dokument WHERE PMID = '" + str(PMID)+ "'")
    return cur.fetchall()[0][0]

def DB_Select_Titel(PMID):
    cur.execute("SELECT Titel FROM Dokument WHERE PMID = '" + str(PMID)+ "'")
    return cur.fetchall()[0][0]


# Funktion, die prüft, ob MeSH Daten schon in DB
def pruefe_MeSH_in_DB(UI):
    cur.execute("SELECT count(1) FROM MeSH WHERE UI = '" + str(UI) + "'")
    Mesh_in_DB = cur.fetchall()[0][0]
    if Mesh_in_DB != 0:
        return 1 # Mesh ist schon in DB
    else:
        return 0 # Mesh ist noch nicht in DB

# Funktion, die prüft, ob Dokument schon in DB
def pruefe_Dok_in_DB(PMID):
    cur.execute("SELECT count(1) FROM Dokument WHERE PMID = '" + str(PMID) + "'")
    Dok_in_DB = cur.fetchall()[0][0]
    if Dok_in_DB != 0:
        return 1 # Dokument ist schon in DB
    else:
        return 0 # Dokument ist noch nicht in DB
    
def DB_Select_MeSH_UI(term):
    cur.execute("SELECT UI FROM MeSH WHERE Term = '" + str(term) + "'")
    MeSH_UI =  cur.fetchall()[0][0]
    return MeSH_UI

In [51]:
# Funktion zur Visualsierung der kumulierten Zitationszahlen
# Ausgabe von Eckdaten

def visualisiereGesamtUeberblick():
    try:
        cur.execute("SELECT count(UI) FROM MeSH")
        count_mesh = cur.fetchall()[0][0]
        x = ["Emergent", "Nicht-Emergent", "Nicht-Emergent Korrigiert"]
        fig = plt.figure(figsize=(15, 10))
        sub1 = plt.subplot(2, 2, 1)
        sub2 = plt.subplot(2, 2, 2)
        # Daten Plot 1
        cur.execute("SELECT sum(EK), sum(NEK), sum(NEKK) FROM MeSH")
        data = cur.fetchall()[0]
        y1 = {"gesamt": [data[0], data[1], data[2]]}
        df1=pd.DataFrame(y1,index=x)
        maxZitation = max(y1["gesamt"])
        # Daten Plot 2
        cur.execute("SELECT EK, NEK, NEKK, UI FROM MeSH")
        data = cur.fetchall()
        y2 = {}
        for ek, nek, nekk, ui in data:
            y2[ui] = [ek, nek, nekk]
        df2=pd.DataFrame(y2,index=x)
        # Plot 
        df1.plot(ax=sub1,kind="bar",stacked=True,legend=None)
        df2.plot(ax=sub2,kind="bar",stacked=True,legend=None)
        
        for i in range(len(x)):
            sub1.text(i, y1["gesamt"][i]+0.015*maxZitation, y1["gesamt"][i], ha = 'center')
            sub2.text(i, y1["gesamt"][i]+0.015*maxZitation, y1["gesamt"][i], ha = 'center')
        sub1.set_title("Gesamtüberblick")
        sub2.set_title("Anzahl analystierter MeSH: " + str(count_mesh))
        sub1.tick_params("x", labelrotation=45)
        sub1.set_ylabel("Anzahl Zitationen")
        sub2.tick_params("x", labelrotation=45)
        sub2.set_ylabel("Anzahl Zitationen")
        sub2.legend(loc="upper right", bbox_to_anchor=(1.3, 1), title="MeSH UI")
        plt.plot()
    except:
        mr.Markdown(f"## Keine Daten vorhanden")

def visualisiereMeSHZeitstrahl(meshUI):
    try:
        fig = plt.figure(figsize=(15, 10))
        sub1 = plt.subplot(2, 2, 1)
        sub2 = plt.subplot(2, 2, 2)
        cur.execute("SELECT Jahr_MeSH FROM MeSH WHERE UI = '" + str(meshUI) + "' ")
        mesh_jahr = cur.fetchall()[0][0]
        # Plot 1
        cur.execute("SELECT Jahr, sum(Anzahl) FROM MeSH_Dok JOIN Zitationen ON MeSH_Dok.PMID = Zitationen.PMID WHERE UI = '" + str(meshUI) + "' GROUP BY UI,Jahr")
        data = cur.fetchall()
        x1 = [dat[0] for dat in data]
        y1 = [dat[1] for dat in data]
        df1 = pd.DataFrame(y1,index=x1)
        df1.plot(ax=sub1,kind="bar",stacked=True,legend=None)
        sub1.tick_params("x", labelrotation=45)
        sub1.set_title("Anzahl der Zitationen für das MeSH: " + str(meshUI) + " \n Aufnahmejahr: " + str(mesh_jahr))
        sub1.set_xlabel("Jahr")
        sub1.set_ylabel("Anzahl Zitationen")
        # Plot 2
        cur.execute("SELECT DISTINCT(Jahr) FROM MeSH_Dok JOIN Zitationen ON MeSH_Dok.PMID = Zitationen.PMID WHERE UI = '" + str(meshUI) + "' ORDER BY Jahr")    
        jahre = [x[0] for x in cur.fetchall()]
        cur.execute("SELECT DISTINCT(Zitationen.PMID) FROM MeSH_Dok JOIN Zitationen ON MeSH_Dok.PMID = Zitationen.PMID WHERE UI = '" + str(meshUI) + "' ")    
        pmids = [x[0] for x in cur.fetchall()]
        data = {}
        for pmid in pmids:
            data[pmid] = np.zeros(len(jahre))
        df2 = pd.DataFrame(data, index=jahre)
        cur.execute("SELECT Zitationen.PMID, Jahr, Anzahl FROM MeSH_Dok JOIN Zitationen ON MeSH_Dok.PMID = Zitationen.PMID WHERE UI = '" + str(meshUI) + "'  ORDER BY Jahr")        
        data = cur.fetchall()
        for pmid, jahr, anzahl in data:
            df2.loc[jahr, pmid] = anzahl
        df2.plot(ax=sub2,kind="bar",stacked=True,legend=None)
        sub2.tick_params("x", labelrotation=45)
        if len(pmids) > 15:
            columns = round(len(pmids)/15)
            sub2.legend(loc=(1.01, 0.01), title="Dokument PMID", ncols=columns)
        else:
            sub2.legend(loc="upper right", bbox_to_anchor=(1.3, 1), title="Dokument PMID")
        sub2.set_title("Anzahl der Zitationen für das MeSH: " + str(meshUI) + " pro Dokument\n Anzahl analysierter Dokumente: "+ str(len(pmids)))
        sub2.set_xlabel("Jahr")
        sub2.set_ylabel("Anzahl Zitationen")
        if len(jahre) > 15:
            for label in sub1.xaxis.get_ticklabels()[::2]:
                label.set_visible(False)
            for label in sub2.xaxis.get_ticklabels()[::2]:
                label.set_visible(False)
    except:
        print("Keine Daten verfügbar")



# Funktion zur Visualisierung der kumulierten Zitationsdaten zu einem MeSH
def visualisiereENENEKkumuliert_UI(UI):
    try:
        Term = DB_Select_Term(UI)
        cur.execute("SELECT EK, NEK, NEKK FROM MeSH WHERE UI = '"+ str(UI) + "'")
        y = cur.fetchall()[0]
        x = [0,1,2]
        maxZitation = max(y)
        if maxZitation > 0:
            fig, ax = plt.subplots(layout="constrained")
            ax.yaxis.set_major_locator(MaxNLocator(integer=True))
            ax.xaxis.set_major_locator(MaxNLocator(integer=True))
            ax.set_ylabel("Anzahl Zitationen")
            ax.set_title("MeshUI: " + str(UI) + ", " + str(Term) + "\n Gesamtanzahl Zitationen: " + str(sum(y)))
            ax.set_xticks(ticks=x, labels=["Emergent", "Nicht-Emergent", "Nicht-Emergent Korrigiert"], rotation=45)
            ax.set_ylim(0, maxZitation+0.1*maxZitation)
            ax.bar(x, y)
            for i in range(len(x)):
                plt.text(i, y[i]+0.02*maxZitation, y[i], ha = 'center')
            ax.plot()
    except:
        print("Keine Daten vorhanden")

# Funktion zur Visualsierung der Zitationen (E, NE, NEK) für ein bestimmtes Dokument zu einem bestimmten MeSH
def visualisiereENENEKkumuliert_UIPMID(UI, PMID):
    try:
        Term = DB_Select_Term(UI)
        Titel = DB_Select_Titel(PMID)
        cur.execute("SELECT EK_Dok, NEK_Dok, NEKK_Dok FROM MeSH_Dok WHERE UI = '"+ str(UI) + "' and PMID = '" + str(PMID) + "'")
        y = cur.fetchall()[0]
        x = [0,1,2]
        maxZitation = max(y)
        if maxZitation > 0:
            fig, ax = plt.subplots(layout="constrained")
            ax.yaxis.set_major_locator(MaxNLocator(integer=True))
            ax.xaxis.set_major_locator(MaxNLocator(integer=True))
            ax.set_ylabel("Anzahl Zitationen")
            ax.set_title("MeshUI: " + str(UI) + ", " + str(Term) + "\n PMID: " +str(PMID) + ", " + str(Titel))
            ax.set_xticks(ticks=x, labels=["Emergent", "Nicht-Emergent", "Nicht-Emergent Korrigiert"], rotation=45)
            ax.set_ylim(0, maxZitation+0.1*maxZitation)
            ax.bar(x, y)
            for i in range(len(x)):
                plt.text(i, y[i]+0.02*maxZitation, y[i], ha = 'center')
            ax.plot()
    except:
        print("Keine Daten vorhanden")

# Funktion zur Visualisierung der Veröffentlichung, Mesh Einführung und Zitationshäufigkeiten
def visualisiereZitationenUndVeröffentlichung(PMID, UI):
    try:
        cur.execute("SELECT Jahr, Anzahl FROM Zitationen WHERE PMID = '" + str(PMID)+ "'")
        data = cur.fetchall()
        if data == []:
            mr.Markdown(f"### Es liegen keine Zitationsdaten für das Dokument mit der PMID: " + str(PMID) + " vor.")
        else:
            try:
                Jahr = [x[0] for x in data]
                Anzahl = [x[1] for x in data]
                maxZitation = max(Anzahl)
                # print(int(0.2*maxZitation))
                cur.execute("SELECT Jahr_Dok FROM Dokument WHERE PMID = '" + str(PMID)+ "'")
                data = cur.fetchall()
                Jahr_Dok = data[0][0]
                cur.execute("SELECT Jahr_MeSH FROM MeSH WHERE UI = '" + str(UI)+ "'")
                data = cur.fetchall()
                Jahr_MeSH = data[0][0]
                if len(Jahr) != 0:
                    fig, ax = plt.subplots(layout="constrained")
                    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
                    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
                    ax.set_ylabel("Anzahl Zitationen")
                    ax.set_xlabel("Jahre")
                    ax.set_title("PMID: " + str(PMID) + " UI: " + str(UI) + "\n Gesamtanzahl Zitationen: " + str(sum(Anzahl)))
                    ax.set_xlim(min(Jahr_Dok, Jahr_MeSH)-5, 2023)
                    ax.set_ylim(0, maxZitation+0.1*maxZitation)
                    ax.bar(Jahr, Anzahl)
                    ax.annotate("Veröffentlichungsjahr",
                        xy=(Jahr_Dok, int(0.2*maxZitation)), 
                        xytext=(0, 0),
                        textcoords="offset points", 
                        arrowprops=dict(facecolor="blue"))
                    ax.annotate("MeSH Aufnahme",
                        xy=(Jahr_MeSH, int(0.4*maxZitation)), 
                        xytext=(0, 0),
                        textcoords="offset points", 
                        arrowprops=dict(facecolor="red"))
                    ax.plot()
            except:
                print("Keine Daten vorhanden")
    except:
        print("Es ist ein Fehler aufgetreten.")

def visualisiereKorrelationen():
    # Plot 1: Je mehr emergente Zitate, desto mehr Zitate insgesamt
    # Merkmal X
    cur.execute("SELECT EK_Dok FROM MeSH_Dok WHERE EK_Dok IS NOT NULL")
    X = [x[0] for x in cur.fetchall()] # Einzelne emergente Zitationszahlen je Dokument

    # Merkmal Y
    cur.execute("SELECT EK_Dok+NEK_Dok FROM MeSH_Dok WHERE EK_Dok IS NOT NULL")
    Y = [y[0] for y in cur.fetchall()] # Einzelne gesamte Zitationszahlen je Dokument

    ergebnis = stats.pearsonr(X,Y)
    cor = ergebnis[0]
    # p_wert = ergebnis[1]
    if cor > 0: 
        print("Es liegt ein positiver linearer Zusammenhang zwischen emergenten und der Gesamtanzahl an Zitationen vor. Korrelationskoeffizient: ", cor)
    if cor == 0: 
        print("Es liegt ein neutraler linearer Zusammenhang zwischen emergenten und der Gesamtanzahl an Zitationen vor. Korrelationskoeffizient: 0")
    if cor < 0: 
        print("Es liegt ein negativer linearer Zusammenhang zwischen emergenten und der Gesamtanzahl an Zitationen vor. Korrelationskoeffizient: ", cor)
    # if p_wert < 0.05:
    #     print("Der p-Wert ist kleiner als das gewählte Signifikanzniveau von 0,05. Damit ist die Korrelation statistisch signifikant. p-Wert: ", p_wert)
    # if p_wert > 0.05:
    #     print("Der p-Wert ist größer als das gewählte Signifikanzniveau von 0,05. Damit ist die Korrelation nicht statistisch signifikant. p-Wert: ", p_wert)
    # Plot 2: Wenn MeSH aufgenommen: dann Anstieg von Zitationszahlen, Verwendung korrigierter nicht-emergenter Zeitraum
    # Merkmal X
    cur.execute("SELECT EK_Dok FROM MeSH_Dok WHERE EK_Dok IS NOT NULL")
    X = [x[0] for x in cur.fetchall()] # Einzelne emergente Zitationszahlen je Dokument

    # Merkmal Y
    cur.execute("SELECT NEKK_Dok FROM MeSH_Dok WHERE EK_Dok IS NOT NULL")
    Y = [y[0] for y in cur.fetchall()] # Einzelne korrigiert nicht-emergente Zitationszahlen je Dokument

    ergebnis = stats.pearsonr(X,Y)
    cor = ergebnis[0]
    # p_wert = ergebnis[1]
    if cor > 0: 
        print("Es liegt ein positiver linearer Zusammenhang zwischen emergenten und korrigierten nicht-emergenten Zitationen vor. Korrelationskoeffizient: ", cor)
    if cor == 0: 
        print("Es liegt ein neutraler linearer Zusammenhang zwischen emergenten und korrigierten nicht-emergenten Zitationen vor. Korrelationskoeffizient: 0")
    if cor < 0: 
        print("Es liegt ein negativer linearer Zusammenhang zwischen emergenten und korrigierten nicht-emergenten Zitationen vor. Korrelationskoeffizient: ", cor)
    # if p_wert < 0.05:
    #     print("Der p-Wert ist kleiner als das gewählte Signifikanzniveau von 0,05. Damit ist die Korrelation statistisch signifikant. p-Wert: ", p_wert)
    # if p_wert > 0.05:
    #     print("Der p-Wert ist größer als das gewählte Signifikanzniveau von 0,05. Damit ist die Korrelation nicht statistisch signifikant. p-Wert: ", p_wert)
    

def visualisiereMeSHAnstieg():
    cur.execute("SELECT count(PMID) FROM MeSH_Dok WHERE EK_Dok < NEKK_DOK")
    mehr = cur.fetchall()[0][0]
    cur.execute("SELECT count(PMID) FROM MeSH_Dok WHERE EK_Dok > NEKK_DOK")
    weniger = cur.fetchall()[0][0]
    cur.execute("SELECT count(PMID) FROM MeSH_Dok WHERE EK_Dok = NEKK_DOK")
    gleich = cur.fetchall()[0][0]
    cur.execute("SELECT count(PMID) FROM MeSH_Dok WHERE EK_Dok = NEKK_DOK and EK_Dok = 0 and NEK_Dok = 0 and NEKK_Dok = 0")
    null = cur.fetchall()[0][0]
    mr.Markdown(f"### Bei " + str(mehr) + " Dokumenten steigt die Anzahl der Zitationen nach der Einführung des MeSH an. (Betrachtung des korrigierten Zeitraums)")
    mr.Markdown(f"### Bei " + str(weniger) + " Dokumenten nimmt die Anzahl der Zitationen nach der Einführung des MeSH ab. (Betrachtung des korrigierten Zeitraums)")
    mr.Markdown(f"### Bei " + str(gleich) + " Dokumenten bleibt die Anzahl der Zitationen nach der Einführung des MeSH gleich. Davon werden " + str(null) + " Dokumente nie zitiert.")


def visualisiereRelativeVerlaufUndMeSH(UI):
    cur.execute("SELECT UI FROM MeSH")
    meshs = [x[0] for x in cur.fetchall()]
    max_anzahl_jahre = 0
    dataframes = [] 
    for meshUI in meshs:
        data_plot = {}
        cur.execute("SELECT Jahr_MeSH FROM MeSH WHERE UI = '" + str(meshUI) + "' ")
        mesh_jahr = cur.fetchall()[0][0]
        cur.execute("SELECT min(Jahr) FROM MeSH_Dok JOIN Zitationen ON MeSH_Dok.PMID = Zitationen.PMID WHERE UI = '" + str(meshUI) + "'")
        min_jahr = cur.fetchall()[0][0]    
        jahr_korrektur = mesh_jahr+mesh_jahr-min_jahr

        anzahl_jahre = mesh_jahr-min_jahr
        if anzahl_jahre > max_anzahl_jahre:
            max_anzahl_jahre = anzahl_jahre
        if anzahl_jahre <= 0:
            continue
        
        data_plot[meshUI] = np.zeros(2*(mesh_jahr-min_jahr)+1)
        jahre = np.linspace(min_jahr, jahr_korrektur, 2*(mesh_jahr-min_jahr)+1,dtype='int')
        df1 = pd.DataFrame(data_plot, index=jahre)
        df1.loc[:, meshUI] = np.zeros(2*(mesh_jahr-min_jahr)+1)
        cur.execute("SELECT Jahr, sum(Anzahl) FROM MeSH_Dok JOIN Zitationen ON MeSH_Dok.PMID = Zitationen.PMID WHERE UI = '" + str(meshUI) + "' and Jahr <= "+str(jahr_korrektur)+" GROUP BY UI,Jahr")
        data = cur.fetchall()
        x1 = [dat[0] for dat in data]
        y1 = [dat[1] for dat in data]
        for dat in data:
            df1.loc[dat[0]][meshUI] = dat[1]/sum(y1)
        index_list_jahre = df1.index.tolist()
        for index in index_list_jahre:
            df1.rename(index={index:index-index_list_jahre[math.floor(len(index_list_jahre)/2)]},inplace=True)
        dataframes.append(df1)
    dataframe = None
    for i in range(len(dataframes)):
        if i == len(dataframes)-1:
            break
        else:
            if len(dataframes[i].index) > len(dataframes[i+1].index):
                dataframe = dataframes[i].join(dataframes[i+1])
            else:
                dataframe = dataframes[i+1].join(dataframes[i])
    fig = plt.figure(figsize=(15, 10))
    sub1 = plt.subplot(2, 2, 1)
    sub2 = plt.subplot(2, 2, 2)
    dataframe.plot(ax=sub2)
    sub2.set_title("Relative Zitationen, vor und nach MeSH-Aufnahme")
    sub2.set_xlabel("Jahre vor (-) und nach (+) MeSH-Aufnahme") 
    sub2.set_ylabel("Relative Anzahl Zitationen\n bezogen auf die Gesamtzahl im korrigierten Zeitraum") 
    sub2.legend(loc="upper right", bbox_to_anchor=(1.3, 1), title="MeSH UI")

    # Plot 2
    try:
        Term = DB_Select_Term(UI)        
        x = ["Emergent", "Nicht-Emergent", "Nicht-Emergent Korrigiert"]
        cur.execute("SELECT EK, NEK, NEKK FROM MeSH WHERE UI = '"+ str(UI) + "'")
        data = cur.fetchall()[0]
        y1 = {"ui": [data[0], data[1], data[2]]}
        df1=pd.DataFrame(y1,index=x)
        maxZitation = max(y1["ui"])
        df1.plot(ax=sub1,kind="bar",stacked=True,legend=None)        
        sub1.tick_params("x", labelrotation=45)
        sub1.set_ylabel("Anzahl Zitationen")
        sub1.set_title("MeshUI: " + str(UI) + ", " + str(Term) + "\n Gesamtanzahl Zitationen: " + str(sum(y1["ui"])))
        for i in range(len(x)):
            sub1.text(i, y1["ui"][i]+0.015*maxZitation, y1["ui"][i], ha = 'center')
        
    except:
        print("Keine Daten vorhanden")




In [ ]:
show_code = mr.Checkbox(label="Quelltext ein-/ausblenden", value=True)
app = mr.App(title="Prozess: Visualisierung", 
             description="Visualisierung der Analyseergebnisse", show_code=show_code.value)
mr.Markdown(f"# Visualisierung von Zitationsdaten")

In [ ]:
alleMeSH_DB = DB_Select_Alle_MeSH()
mesh_auswahl_term = mr.Select(value=alleMeSH_DB[0], choices=alleMeSH_DB, label="Wählen Sie ein MeSH aus:")
mesh_auswahl_ui = DB_Select_MeSH_UI(mesh_auswahl_term.value)
# doks_titel_pmid_json, doks_titel = DB_Select_Alle_Dokumente_MeSH(mesh_auswahl_ui)
# dok_auswahl_titel = mr.Select(value=doks_titel[0], choices=doks_titel, label="Die in Verbindung stehenden Dokumente sind:")
# doks_pmid = doks_titel_pmid_json[dok_auswahl_titel.value]


In [ ]:
_ = visualisiereGesamtUeberblick()
_ = visualisiereMeSHZeitstrahl(mesh_auswahl_ui)
_ = visualisiereRelativeVerlaufUndMeSH(mesh_auswahl_ui)
_ = visualisiereMeSHAnstieg()